# Graph convolutional network

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch_geometric import datasets, transforms

from gutils import GCNNodeClassifier, train

## Dataset

In [ ]:
# load dataset with node features and node labels
data_set = datasets.Planetoid(
    root='../data',
    name='Cora',
    # transform=transforms.NormalizeFeatures(attrs=['x']) # normalize rows
)

In [ ]:
# print summaries
print(f'Number of data points: {len(data_set)}')
print(f'Tensor shapes: {data_set[0]}')

print(f'Number of features: {data_set.num_features}')
print(f'Number of classes: {data_set.num_classes}')

print(f'Number of train nodes: {data_set.train_mask.sum()}')
print(f'Number of val. nodes: {data_set.val_mask.sum()}')
print(f'Number of test nodes: {data_set.test_mask.sum()}')

## Model

In [ ]:
# create model
model = GCNNodeClassifier(
    # num_channels=[data_set.num_features, 128, data_set.num_classes],
    num_channels=[data_set.num_features, 64],
    num_classes=data_set.num_classes
)

In [ ]:
# check output shape
y = model(data_set[0].x, data_set[0].edge_index)

print(f'Prediction shape: {y.shape}')

## Training

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean')

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.01,
    weight_decay=0.01
)

In [ ]:
history = train(
    data=data_set[0],
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=200,
    log_every=1
)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 3.5))

axes[0].plot(
    np.arange(history['num_epochs'] + 1),
    np.asarray(history['train_loss']),
    label='train', alpha=0.7
)
axes[0].plot(
    np.arange(history['num_epochs'] + 1),
    np.asarray(history['val_loss']),
    label='val.', alpha=0.7
)
axes[0].set(xlabel='epoch', ylabel='loss')

axes[1].plot(
    np.arange(history['num_epochs'] + 1),
    np.asarray(history['train_acc']),
    label='train', alpha=0.7
)
axes[1].plot(
    np.arange(history['num_epochs'] + 1),
    np.asarray(history['val_acc']),
    label='val.', alpha=0.7
)
axes[1].set(xlabel='epoch', ylabel='acc.')
axes[1].yaxis.set_label_position('right')
axes[1].yaxis.tick_right()

for ax in axes:
    ax.set_xlim((0, history['num_epochs']))
    ax.legend()
    ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax.set_axisbelow(True)

fig.tight_layout()